In [15]:
import pyodbc
import re
import logging

# User variables

In [16]:
# 231: Zoopla Capital
# 232: Zoopla Rental
# n.b. 231 covers both datasets in SDR, so 231 is hard-coded in some places. This is not a mistake.
PROPERTY_SOURCE_ID = 232

COMPS_SERVER = r"LON-SQL-04"
SDR_SERVER = r"LON-SQL-01\SOURCEBUILD"

# Set system variables

In [17]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

COMPSBUILD_CONNECTION_STRING = f"Server={COMPS_SERVER};Database=Logging;Driver={{SQL Server}};Trusted Connection=True"
SOURCEBUILD_CONNECTION_STRING = f"Server={SDR_SERVER};Database=Logging;Driver={{SQL Server}};Trusted Connection=True"

if PROPERTY_SOURCE_ID == 231:
    source_table = "sdr.staging.tab_231_CapitalListings_ForComps"
    price_column = "askingPrice"
elif PROPERTY_SOURCE_ID == 232:
    source_table = "sdr.staging.tab_231_RentalListings_ForComps"
    price_column = "MarketRent"
else:
    raise Exception("invalid PROPERTY_SOURCE_ID")

# Find out what data was imported for the last build

In [18]:
db = pyodbc.connect(COMPSBUILD_CONNECTION_STRING)

query = """
SELECT TOP 1 sourceLocation
FROM comparablesImport.dbo.tab_importBatch ib
WHERE propertysourceid = ?
ORDER BY dtmimported DESC
"""

with db.cursor() as cursor:
    cursor.execute(query, PROPERTY_SOURCE_ID)
    row = cursor.fetchone()

db.close()

In [19]:
regex = r"WHERE importBatchId>[0-9]{0,} and importBatchId<=([0-9]{0,})"
m = re.search(regex, row[0])
previous_end = m.groups(0)[0]

logging.info(f"latest previous batch was to {previous_end}")

INFO:root:latest previous batch was to 310


# Find out what data needs to be imported this build

In [20]:
db = pyodbc.connect(SOURCEBUILD_CONNECTION_STRING)

query = """
SELECT  
    [dataSourceId]
    ,[importVersion]
    ,[importBatchId]
    ,[importBatchObject]
    ,[rowsImported]
    ,[batchActivationDate]
FROM [SDR].[imp].[tab_dataSourceImportBatch]
WHERE datasourceId=231
and importVersion=1
AND importBatchId > ?
ORDER BY batchActivationDate
"""

with db.cursor() as cursor:
    cursor.execute(query, previous_end)
    rows = cursor.fetchall()

db.close()

In [21]:
batches_to_import = [x[2] for x in rows]

if len(batches_to_import) == 0:
    logging.warning("nothing to import")

current_start = min(batches_to_import)
current_end = max(batches_to_import)
logging.info(f"new batches are {current_start} to {current_end}")

INFO:root:new batches are 311 to 314


# Get the comps importRunID for this build

In [22]:
db = pyodbc.connect(COMPSBUILD_CONNECTION_STRING)

query = """
select TOP 1 *
from comparablesImport.dbo.tab_importRun
ORDER BY createDate desc
"""

with db.cursor() as cursor:
    cursor.execute(query)
    row = cursor.fetchone()

db.close()

In [23]:
comps_run_id = row[0]
logging.info(f"comps build importRunId is {comps_run_id}")

INFO:root:comps build importRunId is 124


# Create a new tab_propertyRaw batchId

In [24]:
db = pyodbc.connect(COMPSBUILD_CONNECTION_STRING)

query = f"""
DECLARE @sourceLocation VARCHAR(1000)
= 'SELECT * FROM sdr.raw.tab_231_1 AS [raw] 
    INNER JOIN {source_table} AS cl
    ON [raw].rawRowId = cl.rawRowId        
    AND cl.importVersion = 1
    WHERE importBatchId>{current_start - 1} and importBatchId<={current_end}'
,   @importRunId INT = ?
EXEC comparablesImport.[dbo].[proc_createNewImportBatch]
    @propertysourceid = ?
    , @sourcetype = ?
    , @sourcelocation = @sourceLocation
    , @importRunId  = @importRunId
"""

with db.cursor() as cursor:
    cursor.execute(query, comps_run_id, PROPERTY_SOURCE_ID, source_table)
    row = cursor.fetchone()

db.close()

In [25]:
property_raw_batch = row[0]
logging.info(f"tab_propertyRaw batch is {property_raw_batch}")

INFO:root:tab_propertyRaw batch is 66


# Import the data into tab_propertyRaw

In [26]:
db = pyodbc.connect(SOURCEBUILD_CONNECTION_STRING)

query = f"""
set xact_abort on;

INSERT INTO [{COMPS_SERVER}].comparablesBuild.core.tab_propertyRaw ( 
    AddresspointToid
    , batchId
    , blnDeleted
    , Comments
    , custRef1
    , fulladdress
    , LastUpdateDate
    , instrType1
    , bathrooms
    , bedrooms
    , NoOfFloors
    , receptions
    , postcode
    , {price_column}
    , propStyle
    , propertySourceId
    , custRef2
    , tenure
    , yearBuilt
)
SELECT cl.AddressPointToid
    , batchId = ?
    , blnDeleted = 0
    , comments = CONCAT('[fileId]=',CAST(raw.importBatchId AS NVARCHAR(10)),';[rowId]=',CAST(cl.rawRowId AS NVARCHAR(20)),';[listingId]=',CAST(cl.listing_id AS NVARCHAR(10)))
    , custRef1 = cl.CycleNumber
    , fullAddress = cl.full_address
    , lastUpdateDate = cl.listing_date
    , instrType1 = cl.listing_status
    , bathrooms = cl.num_baths
    , bedrooms = cl.num_beds
    , NoOfFloors = cl.num_floors
    , receptions = cl.num_recepts
    , postcode = cl.postcode
    , askingPrice = cl.price
    , propStyle = cl.property_type
    , propertySourceId = ?
    , CustRef2 = cl.sequence_number
    , tenure = cl.tenure
    , yearBuilt = CASE WHEN cl.new_home = 'true' THEN YEAR(cl.listing_date) ELSE NULL END
FROM sdr.raw.tab_231_1 AS [raw]
INNER JOIN {source_table} AS cl 
    ON [raw].rawRowId = cl.rawRowId
    and cl.importVersion = 1
WHERE 
    importBatchId > ?
    and importBatchID <= ?
"""

with db.cursor() as cursor:
    res = cursor.execute(query, property_raw_batch, PROPERTY_SOURCE_ID, current_start - 1, current_end)

db.close()

# Update meta data

In [27]:
rows_inserted = res.rowcount

In [28]:
db = pyodbc.connect(COMPSBUILD_CONNECTION_STRING)

query = f"""
UPDATE ib
SET 
    rawrowsimported = {rows_inserted}
    , apmatchcount = {rows_inserted} -- we only import address matched data
FROM comparablesImport.dbo.tab_importBatch as ib
WHERE 
    propertysourceid = ?
    and batchId = {property_raw_batch}
    and importRunId = {comps_run_id}
"""

with db.cursor() as cursor:
    res = cursor.execute(query, PROPERTY_SOURCE_ID)

db.close()

logging.info(f"{res.rowcount} records updated")

INFO:root:1 records updated
